In [58]:
data = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_20221231\\소상공인시장진흥공단_상가(상권)정보_서울_202212.csv")
data2=data[['상호명','도로명주소','상권업종대분류명','상권업종중분류명','상권업종소분류명','표준산업분류명','행정동명','위도','경도']]
data2_food = data2[data2['상권업종대분류명']=='음식']
data2_food.columns = ['name','address','category1','category2','category3','classifi_name','dong','위도','경도']   

In [59]:
def delete(df):
    df = df.drop(df[(df.category3.str.contains('구내식당'))].index).reset_index(drop=True)
    
    # 카테고리 1은 모두 음식이므로 삭제
    df = df.drop(columns='category1')
    
    df = df.drop(df[(df.category2.str.contains('음식배달'))].index).reset_index(drop=True)
    
    df[df.classifi_name.isnull()] = df[df.classifi_name.isnull()].fillna('부페 음식점업')
    
    df = df.reset_index(drop=True)
    
    return df

data2_food = delete(data2_food)

In [60]:
data2_food

,name,address,category2,category3,classifi_name,dong,위도,경도
0,비지트,서울특별시 서초구 동광로18길 82,한식,한식/백반/한정식,한식 음식점업,방배4동,37.488375,126.991394
1,쓰리에프,서울특별시 동작구 동작대로27가길 12,한식,한식/백반/한정식,한식 음식점업,사당2동,37.487105,126.980952
2,다향,서울특별시 서초구 효령로 230,한식,한식/백반/한정식,한식 음식점업,서초3동,37.483436,127.009382
3,고향생막걸리,서울특별시 강남구 압구정로 216,유흥주점,민속주점,기타 주점업,압구정동,37.528073,127.031012
4,도전최강달인왕만두,서울특별시 강동구 고덕로 353,한식,한식/백반/한정식,한식 음식점업,고덕2동,37.556197,127.159471
...,...,...,...,...,...,...,...,...
125347,텐퍼센트커피,서울특별시 금천구 금하로 619,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,시흥1동,37.454688,126.899911
125348,청담우애,서울특별시 강남구 도산대로90길 16,한식,한식/백반/한정식,한식 음식점업,청담동,37.523542,127.048901
125349,막리단길,서울특별시 성북구 동소문로20다길 11,유흥주점,꼬치구이전문점,기타 주점업,동선동,37.591835,127.017312
125350,보름설기,서울특별시 구로구 신도림로11가길 36,제과제빵떡케익,떡전문,제과점업,신도림동,37.506249,126.876330


In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import time
import numpy as np
import pandas as pd



url = ['https://place.map.kakao.com/16821812',
       'https://place.map.kakao.com/1586739582'
'https://place.map.kakao.com/16885409',
'https://place.map.kakao.com/18831509',
'https://place.map.kakao.com/1865670192',
'https://place.map.kakao.com/13055623',
'https://place.map.kakao.com/9610309'
]

driver = webdriver.Chrome() 
df=pd.DataFrame(columns=['url','kakao_store_name','kakao_star_point','kakao_star_point_qty','kakao_blog_review_txt','kakao_blog_review_qty'])

df['url']=url

# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

kakao_map_name_list = []
blog_review_list = []
blog_review_qty_list = []
kakao_map_star_review_stars_list = []
kakao_map_star_review_qty_list = []

for i, url in enumerate(df['url']):
    driver.get(url)
    time.sleep(1)
    review_text = ""
    
    try:
        
        kakao_map_name = driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2").text
        blog_review_qty = driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(5) > span").text
        star_review_stars = driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_b.inactive").text
        star_review_qty = driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_g").text
    except Exception as e1:
        if "inactive" in str(e1):
            star_review_stars = driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_b").text
            star_review_qty = driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_g").text
        else:
            print(f"{num}행 데이터의 평점 갯수 크롤링에 다른 문제가 생겼다")
            
    num = 0
    try:
        while num < 3:
            try:
                num += 1
                review_text = review_text + driver.find_element(By.CSS_SELECTOR,f"div.wrap_list > ul > li:nth-child({num}) > a > div.review_story > p").text
                
                if num == 3:
                    blog_review_list.append(review_text)  # 3개나 찾았으면 그만 찾고 나감
                    kakao_map_name_list.append(kakao_map_name)
                    blog_review_qty_list.append(blog_review_qty)
                    kakao_map_star_review_stars_list.append(star_review_stars)
                    kakao_map_star_review_qty_list.append(star_review_qty)
                    
                    break
            except Exception as e1:
                if "li:nth-child(1)" in str(e1):  # child(1)이 없던데요? -> 리뷰가 하나도 없는 것
                    print(f"{i}행은 리뷰가 없네")
                    review_text = "empty"
                    blog_review_list.append(review_text)
                    kakao_map_name_list.append(kakao_map_name)
                    blog_review_qty_list.append(blog_review_qty)
                    kakao_map_star_review_stars_list.append(star_review_stars)
                    kakao_map_star_review_qty_list.append(star_review_qty)
                    break
                else:
                    print(f"{i}행 문제가 발생 - 리뷰가 {num - 1}개뿐이다")
                    blog_review_list.append(review_text)  # 일단 리뷰가 하나도 없는 건 아니니 붙이고 탈출 / 리뷰 딸랑 하나 있으면 발생할 수 있음                    
                    kakao_map_name_list.append(kakao_map_name)
                    blog_review_qty_list.append(blog_review_qty)
                    kakao_map_star_review_stars_list.append(star_review_stars)
                    kakao_map_star_review_qty_list.append(star_review_qty)
                    break
                
    except Exception as e2:
        print(e2)
        print(f"{i}행 문제가 발생 - code 2")
driver.quit()


df['kakao_store_name'] = kakao_map_name_list  # 카카오 상세페이지에서 크롤링한 상호명
df['kakao_star_point'] = kakao_map_star_review_stars_list  # 카카오 상세페이지에서 평가한 별점 평점
df['kakao_star_point_qty'] = kakao_map_star_review_qty_list  # 카카오 상세페이지에서 별점 평가를 한 횟수
df['kakao_blog_review_txt'] = blog_review_list  # 카카오 상세페이지에 나온 블로그 리뷰 텍스트들
df['kakao_blog_review_qty'] = blog_review_qty_list  # 카카오 상세페이지에 나온 블로그 리뷰의 총 개수
 
df

0행 문제가 발생 - 리뷰가 1개뿐이다
2행 데이터의 평점 갯수 크롤링에 다른 문제가 생겼다
1행은 리뷰가 없네
2행 문제가 발생 - 리뷰가 2개뿐이다
3행은 리뷰가 없네
4행 문제가 발생 - 리뷰가 1개뿐이다
5행 문제가 발생 - 리뷰가 1개뿐이다


,url,kakao_store_name,kakao_star_point,kakao_star_point_qty,kakao_blog_review_txt,kakao_blog_review_qty
0,https://place.map.kakao.com/16821812,이레식당,0,(0),한식을 사랑하는 배부름!! 직장인과 자취생들을 위한 맛있는 음식과 여러가지 메뉴가 ...,1
1,https://place.map.kakao.com/1586739582https://...,이레식당,0,(0),empty,1
2,https://place.map.kakao.com/18831509,백합반점,5.0,(1),신대방에 위치한 백합반점을 소개합니다. 백합반점은(일명 백합꼬치집) 중국인 사장님이...,2
3,https://place.map.kakao.com/1865670192,대림김밥,0,(0),empty,0
4,https://place.map.kakao.com/13055623,피자스쿨 대방현대점,5.0,(5),안녕하세요 영이에요:) 오늘은 간단하고 맛있는 피자를 먹고 왔는데요! 짜잔 그래서 ...,1
5,https://place.map.kakao.com/9610309,블룸,4.0,(1),블룸 또 갔다믄써~~~ 중대앞에서 칵테일 마시고 싶다면 여기가 ㄱㅊ은듯?? 새벽 1...,1


In [74]:
df['kakao_star_point'] = df['kakao_star_point'].astype('float')
df['kakao_blog_review_qty'] = df['kakao_blog_review_qty'].astype('int')
df['kakao_star_point_qty'] = df['kakao_star_point_qty'].str.replace('(', '').str.replace(')', '').astype(int)

C:\Users\KIMJEONGSU.DESKTOP-3GN3AD1\AppData\Local\Temp\ipykernel_15532\4113528988.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['kakao_star_point_qty'] = df['kakao_star_point_qty'].str.replace('(', '').str.replace(')', '').astype(int)


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   url                    6 non-null      object 
 1   kakao_store_name       6 non-null      object 
 2   kakao_star_point       6 non-null      float64
 3   kakao_star_point_qty   6 non-null      int32  
 4   kakao_blog_review_txt  6 non-null      object 
 5   kakao_blog_review_qty  6 non-null      int32  
 6   cate_mix               6 non-null      object 
dtypes: float64(1), int32(2), object(4)
memory usage: 416.0+ bytes


In [76]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df['cate_mix'] = data2_food['category2'] +  data2_food['category3']
df['cate_mix'] = df['cate_mix'].str.replace("/", " ")


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]



# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df['kakao_blog_review_txt']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [80]:
# 공식 1~5의 중요성을 짬뽕시키는 공식
# * 0.003 등의 가중치를 줘서 조절합니다.

place_simi_co = (
                 + place_simi_cate * 0.3 # 공식 1. 카테고리 유사도
                 + place_simi_review * 1 # 공식 2. 리뷰 텍스트 유사도
                 + np.repeat([df['kakao_blog_review_qty'].values], len(df['kakao_blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                 + np.repeat([df['kakao_star_point'].values], len(df['kakao_star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                 + np.repeat([df['kakao_star_point_qty'].values], len(df['kakao_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )



# 아래 place_simi_co_sorted_ind 는 그냥 바로 사용하면 됩니다.
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    place_title = df[df['kakao_store_name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]


# 상도국수를 포함해 5개 업체를 뽑아봅시다.

find_simi_place(df, place_simi_co_sorted_ind, '김밥', 1)

,url,kakao_store_name,kakao_star_point,kakao_star_point_qty,kakao_blog_review_txt,kakao_blog_review_qty,cate_mix
